In [ ]:
!nvidia-smi

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
print(tf.__version__)

#1-1 inpiut tensor와 target tensor 준비(훈련데이터)



In [ ]:
auto_mpg_dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
dataset_path = tf.keras.utils.get_file("/content/auto-mpg.data", auto_mpg_dataset_url)
print(dataset_path)

In [ ]:
#자동차 연비 데이처 셋의 컬럼명
column_names = ['MPG','Cylinders','Displacement','Horsepower',
                'Weight','Acceleration','Model Year','Origin']

raw_dataset=pd.read_csv(dataset_path, names=column_names, na_values='?',
            comment='\t', sep=' ', skipinitialspace=True)
dataset = raw_dataset.copy()
dataset

#1-2 입력 데이터의 전처리

##데이터 셋 내의 NaN값 제거

In [ ]:
#데이터 셋 내의 NaN값 체크
dataset.isna().sum()

In [ ]:
#데이터 셋 내의 NaN값 제거
dataset.dropna(inplace=True)


In [ ]:
len(dataset)

In [ ]:
dataset.isna().sum()

##데이터셋에서 20% 선택해 test데이터 셋으로 분할

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
print(len(train_dataset))
print(len(test_dataset))

##표준화를 통한 input tensor의 전처리

In [ ]:
#데이터 전처리를 위한 통계정보 생성
train_stats = train_dataset.describe()

In [ ]:
train_stats

In [ ]:
#정답 데이터(MPG-연비)제거
train_stats.pop('MPG')

In [ ]:
train_stats

In [ ]:
#각 특성 별 통계정보를 가공하기 쉽도록 matrix 전치
train_stats = train_stats.transpose()

In [ ]:
train_stats

In [ ]:
#target tensor를 위한 연비 값(MPG)추출
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [ ]:
#input tensor의 표준화(평균 =0, 표준편차 =1)함수 정의
def norm(x):
  normed_data =(x-train_stats['mean']/train_stats['std'])
  return normed_data

In [ ]:
#input tensor의 특성 정보 표준화 처리 수행
train_dataset = norm(train_dataset)
test_dataset = norm(test_dataset)

In [ ]:
train_dataset.describe()

#2. DNN(MLP) 모델 디자인

In [ ]:
#hidden layer : 2개 [64,32], activation : relu
model = models.Sequential()
model.add(layers.Dense(units = 64, activation='relu',
                       input_shape=(len(train_dataset.keys()),)))
model.add(layers.Dense(units=32, activation='relu'))
model.add(layers.Dense(units=1))

In [ ]:
model.summary()

#3. 모델의 학습 정보 설정

In [ ]:
#loss : MSE/ optimizer : rmsprop/ metric : MAE
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['mae'])

#4. 모델에 input,target 데이터 연결 후 학습

In [ ]:
# batch size : 16, epochs : 100, validation data set percent : 20%
history = model.fit(
    x=train_dataset, y=train_labels,
    batch_size=16, epochs=100,
    validation_split=0.2
)

#학습과정의 시각화 및 성능 테스트

In [ ]:
hist = pd.DataFrame(history.history)
hist

In [ ]:
#학습과정 시각화를 위한 정보 추출
loss = history.history['loss']
val_loss = history.history['val_loss']

mae = history.history['mae']
val_mae = history.history['val_mae']

In [ ]:
# 차트의 x축을 위한 epoch정보 생성
epochs = range(1, len(loss)+1)

In [ ]:
#loss 정보 시각화
plt.plot(epochs,loss,label='Training loss')
plt.plot(epochs,val_loss,label='Validation loss')
plt.title('Training and validation loss')
plt.ylim([0,20])
plt.legend()

plt.show()

In [ ]:
#MAE 정보 시각화
plt.plot(epochs,mae,label='Training mae')
plt.plot(epochs,val_mae,label='Validation mae')
plt.title('Training and validation mae')
plt.xlim([1,100])
plt.ylim([0,5])
plt.legend()

plt.show()

In [ ]:
#테스트 데이터 셋을 통한 성능 측정
test_loss, test_mae = model.evaluate(x=test_dataset, y=test_labels)

In [ ]:
#테스트 데이터 셋의 정답과 모델의 예측값 분석을 위한 모델의 예측값 생성
test_predictions = model.predict(test_dataset).flatten()

In [ ]:
#테스트 데이터 셋의 정답과 모델의 예측값을 이용한 상관관계 시각화
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values [MPG]')
plt.ylabel('Predictions [MPG')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,50])
plt.ylim([0,50])
_ = plt.plot([-100,100],[-100,100])